In [87]:
using GLPK
using QPSReader
using NLPModels
using QuadraticModels
using SparseArrays

In [142]:
function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0,  name=name_pb)

end


function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end


function OptimizeGLPK(qpdata)
    QM = createQuadraticModel(qpdata, "QM")
    SM = SlackModel(QM)
    f = grad(SM, zeros(length(SM.meta.x0)))
    Aeq = jac(SM, SM.meta.x0) 
    beq = SM.meta.lcon
    lb = SM.meta.lvar
    ub = SM.meta.uvar
    m,n = size(Aeq)
    x0 = init_x0(lb, ub)
    
    prev_term_out = GLPK.term_out(GLPK.OFF)
    
    lp = GLPK.Prob()
    GLPK.set_prob_name(lp, "sample")
    GLPK.set_obj_name(lp, "OBJECTIVE")
    GLPK.set_obj_dir(lp, GLPK.MIN)

    GLPK.add_rows(lp, m)
    for i=1:m
        GLPK.set_row_name(lp, i, string("A", i, "x"))
        GLPK.set_row_bnds(lp, i, GLPK.FX, beq[i], beq[i]) # fixed bounds
    end
    GLPK.add_cols(lp, n)
    for i=1:n
        GLPK.set_col_name(lp, i, string("x", i))
        GLPK.set_col_bnds(lp, i, GLPK.DB, lb[i], ub[i]) # double bounded
        GLPK.set_obj_coef(lp, i, f[i])
    end
    ia = zeros(Int, n*m)
    ja = zeros(Int, n*m)
    ar = zeros(n*m)
    for i=1:m
        for j=1:n
            ia[n*(i-1) + j] = i
            ja[n*(i-1) + j] = j 
            ar[n*(i-1) + j] = Aeq[i,j]
        end
    end
    sm = sparse(ia, ja, ar)
    GLPK.load_matrix(lp, sm)

    
    #param = GLPK.SimplexParam()
    #param.presolve = GLPK.ON
    #GLPK.init_smcp(param)
    #param["msg_lev"] = GLPK.MSG_ERR
    #param["presolve"] = GLPK.ON
    #param.msg_lev = GLPK.MSG_ERR
    param = GLPK.InteriorParam()
    
    param.msg_lev = GLPK.MSG_ERR
    #flag = GLPK.simplex(lp, param)
    flag = GLPK.interior(lp, param)
    GLPK.term_out(prev_term_out)
    println(GLPK.get_obj_val(lp))
    return flag   
end


OptimizeGLPK (generic function with 1 method)

In [143]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);


┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [144]:
flag1 = OptimizeGLPK(qpdata2)

Assertion failed: x[j] > 0.0 && z[j] > 0.0
Error detected in file draft/glpipm.c at line 508


GLPKFatalError: GLPKFatalError("GLPK call failed. All GLPK objects you defined so far are now invalidated.")

In [145]:
flag1

10